<a href="https://colab.research.google.com/github/nicoloceneda/Python-edu/blob/master/TensorFlow_Computation_Graphs_and_Gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow - Computation Graphs and Gradients
---



In [0]:
import numpy as np
import tensorflow as tf

## Creating a computation graph
TensorFlow relies on computation graphs to perform computations and derive the relationship between tensors, from the input all the way to the output. A computation graph is a network of nodes, where each node represents a tensor or an operation, which applies a function to its input tensor(s) and returns zero or more output tensors.


### TensorFlow v1.x
TensorFlow v1.x's low-level API uses **static computation graphs**, which must be explicitly defined, although this is not trivial for large and complex models. The steps for building, compiling and evaluating a graph are:

1. Instantiate an empty graph using `tf.Graph()`
2. Add nodes to the computation graph using `g.as_default()`
3. Evaluate the graph: \
     3.1 Start a new session using `tf.compat.v1.Session()` \
     3.2 Initialize the variables using `tf.compat.v1.global_variables_initializer()` \
     3.3 Execute the operations using `sess.run()` \

In [0]:
# Example 1: only constants
g = tf.Graph()

with g.as_default():
  a = tf.constant(1, name='a')
  b = tf.constant(2, name='b')
  c = tf.constant(3, name='c')
  z = 2 * (a - b) + c

with tf.compat.v1.Session(graph=g) as sess:
  print('z =', sess.run(z))

z = 1


In [0]:
# Example 2: only placeholders
g = tf.Graph()

with g.as_default():
  a = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_a')
  b = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_b')
  c = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_c')
  z = 2 * (a - b) + c

with tf.compat.v1.Session(graph=g) as sess:
  print('z =', sess.run(z, feed_dict = {a: 1, b: 2, c: 3}))

z = 1


In [0]:
# Example 3: placeholders and variables
g = tf.Graph()

with g.as_default():
  x = tf.compat.v1.placeholder(shape=None, dtype=tf.float32, name='x')
  w = tf.Variable(2.0, name='weight')
  b = tf.Variable(0.7, name='bias')
  z = w * x + b

  init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session(graph=g) as sess:
  sess.run(init)
  for t in [1.0, 0.6, -1.8]:
    print('x = {:>5.2f} --> z = {:>5.2f}'.format(t, sess.run(z, feed_dict={x: t})))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
x =  1.00 --> z =  2.70
x =  0.60 --> z =  1.90
x = -1.80 --> z = -2.90


### TensorFlow v2
TensorFlow v2 uses **dynamic computation graphs** (also called eager execution), which allow to evaluate operations on the fly.

In [0]:
# Example 1 from TensorFlow v1.x
a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = tf.constant(3, name='c')
z = 2 * (a - b) + c

tf.print('z =', z)

z = 1


In [0]:
# Example 2 from TensorFlow v1.x
def compute_z(a, b, c):
  z = tf.add(tf.multiply(2, tf.subtract(a, b)), c)
  return z

z = compute_z(1, 2, 3)
tf.print('z =', z)

z = 1


In [0]:
# Example 3 from TensorFlow v1.x
def compute_z(x):
  w = tf.Variable(2.0)
  b = tf.Variable(0.7)
  return tf.add(tf.multiply(w, x), b)

x = tf.constant(3.0)
print('z =', compute_z(x).numpy())



z = 6.7


Since dynamic graphs are not as computationally efficient as static ones, TensorFlow v2 automatically compiles Python code into a static graph using a tool called AutoGraph. 

Compile a **function into a static graph** using the `@tf.function` decorator.

In [0]:
@tf.function
def compute_z(a, b, c):
  z = tf.add(tf.multiply(2, tf.subtract(a, b)), c)
  return z

z = compute_z(1, 2, 3)
print('z =', z.numpy())

z = 1


**Limit the ways to call** a function using `tf.TensorSpec`.

In [0]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32), 
                              tf.TensorSpec(shape=[None], dtype=tf.int32), 
                              tf.TensorSpec(shape=[None], dtype=tf.int32),))
def compute_z(a, b, c):
  z = tf.add(tf.multiply(2, tf.subtract(a, b)), c)
  return z

# Calling the function using tensors with rank 1 or lists that can be converted to rank 1 tensors:
tf.print('Rank 1 inputs:', compute_z([1], [2], [3]))
tf.print('Rank 1 inputs:', compute_z([1, 2], [2, 4], [3, 6]))

# Calling the function using tensors with ranks other than 1 will result in errors:
# tf.print('Rank 0 inputs:', compute_z(1, 2, 3)) 
# tf.print('Rank 2 Inputs:', compute_z([[1], [2]], [2], [4]], [[3], [6]]))

Rank 1 inputs: [1]
Rank 1 inputs: [1 2]


To use a **variable inside a decorated function**, define the variable outside of the decorated function.

In [0]:
w = tf.Variable(tf.random.uniform((2, 3)))

@tf.function
def compute_z(x):
  return tf.matmul(w, x)

x = tf.constant([[1], [2], [3]], dtype=tf.float32)
print(compute_z(x).numpy())

[[3.8468504]
 [5.1717687]]


## Computing gradients
Optimizing neural networks via algorithms such as stochastic gradient descent requires computing the gradients of the cost with respect to the weights. TensorFlow supports automatic differentiation, which represents a set of computational techniques for computing derivatives or gradients of arithmetic operations. TensorFlow uses computation graphs to compute gradients.

Compute the **gradient wrt a trainable variable** using `tf.GradientTape`.



In [0]:
w = tf.Variable(1.0, trainable=True)
b = tf.Variable(0.5, trainable=True)

x = tf.constant([1.4])
y = tf.constant([2.1])

with tf.GradientTape() as tape:
  z = tf.add(tf.multiply(w, x), b)
  loss = tf.reduce_sum(tf.square(tf.subtract(y, z)))

dloss_dw = tape.gradient(loss, w)
print('dL/dw =', dloss_dw.numpy())

dL/dw = -0.55999976


Compute the **gradient wrt a non-trainable variable** using `tf.GradientTape` and `tape.watch`.

In [0]:
w = tf.Variable(1.0, trainable=True)
b = tf.Variable(0.5, trainable=True)

x = tf.constant([1.4])
y = tf.constant([2.1])

with tf.GradientTape() as tape:
  tape.watch(x)
  z = tf.add(tf.multiply(w, x), b)
  loss = tf.reduce_sum(tf.square(tf.subtract(y, z)))

dloss_dx = tape.gradient(loss, x)
print('dL/dx =', dloss_dx.numpy())

dL/dx = [-0.39999986]


When we monitor computations with `tf.GradientTape`, by default the tape keeps the resources only for a single gradient computation: after calling `tape.gradient` once, the tape is cleared. 

Compute **more than one gradient** making the tape `persistent`. 



In [0]:
w = tf.Variable(1.0, trainable=True)
b = tf.Variable(0.5, trainable=True)

x = tf.constant([1.4])
y = tf.constant([2.1])

with tf.GradientTape(persistent=True) as tape:
  z = tf.add(tf.multiply(w, x), b)
  loss = tf.reduce_sum(tf.square(tf.subtract(y, z)))

dloss_dw = tape.gradient(loss, w)
print('dL/dw =', dloss_dw.numpy())

dloss_db = tape.gradient(loss, b)
print('dL/db =', dloss_db.numpy())

dL/dw = -0.55999976
dL/db = -0.39999986


**Define an optimizer** and apply the gradients to optimize the model parameters using `tf.keras.optimizers`.

In [0]:
w = tf.Variable(1.0, trainable=True)
b = tf.Variable(0.5, trainable=True)

x = tf.constant([1.4])
y = tf.constant([2.1])

with tf.GradientTape(persistent=True) as tape:
  z = tf.add(tf.multiply(w, x), b)
  loss = tf.reduce_sum(tf.square(tf.subtract(y, z)))

dloss_dw = tape.gradient(loss, w)
dloss_db = tape.gradient(loss, b)

optimizer = tf.keras.optimizers.SGD()
optimizer.apply_gradients(zip([dloss_dw, dloss_db], [w, b]))
print('Updated w:', w.numpy())
print('Updated b:', b.numpy())

Updated w: 1.0056
Updated b: 0.504
